# 02 | Experimentos

In [2]:
import subprocess
import pandas as pd
import numpy as np
import os
from pathlib import Path
from IPython.display import display, clear_output
from random import seed
from random import randint
import multiprocessing

In [3]:
def getAllFilesAsListOfStrings( dir_path ):
    dirPathAsString = dir_path 
    dir_paths = []
    for f in os.listdir(dir_path):
        dir_paths.append( dir_path + f )
    return dir_paths

In [4]:
THREADS = multiprocessing.cpu_count()
result_dir = "archivos"
cantidadArchivos = 500
cantidadPalabrasPorArchivo = 700
ITERACIONES = 5

In [5]:
def generarArchivoUniforme(nombreArchivo, largoPalabra, cantidadPalabras):
    os.makedirs(F"../data/uniforme/", exist_ok=True)
    pathArchivo = "../data/uniforme/" + nombreArchivo
    archivo = open(pathArchivo, "w")
    sal = ""
    largoPalabra = largoPalabra
    for i in range(cantidadPalabras):
        for lp in range(largoPalabra):
            sal += chr(randint(97, 122))
        archivo.write(sal + "\n")
        sal = ""
    return pathArchivo

def generarCarpetaUniformes(cantArchivos, cantLetras, cantidadPalabras):
    
    if not os.path.exists("../data/uniforme/"):
        os.makedirs("../data/uniforme/")

    #borramos los archivos viejos para generar los nuevos
    for f in os.listdir("../data/uniforme/"):
        os.remove("../data/uniforme/"+ str(f) )

    fileName = "uniforme"
    for i in range(cantArchivos):
        nombre = fileName + str(i)
        generarArchivoUniforme(nombre, cantLetras, cantidadPalabras)

In [6]:
def generarArchivoNormal(nombreArchivo, largoPalabra, cantidadPalabras, _sigma):
    # a b c d e f g h i j k l m 13 n o p q r s t u v w x y z
    mu= 109.5 #normal( 109.5, 3 ) 
    sigma = _sigma
    #con estos valores, la probabilidad de generar un valor fuera del rago de los caracteres ascii es demasiado baja.
    #de todas formas si se generara alguno, lo descartamos antes de convertirlo a char
    os.makedirs(F"../data/normal/", exist_ok=True)
    pathArchivo = "../data/normal/" + nombreArchivo
    archivo = open(pathArchivo, "w")
    sal = ""
    largoPalabra = largoPalabra
    for i in range(cantidadPalabras):
        for lp in range(largoPalabra):
            val = int(np.random.normal()*sigma + mu)
            if val >=97 and val <= 122:
                sal += chr(val)
        archivo.write(sal + "\n")
        sal = ""
    archivo.close()
    return pathArchivo

def generarCarpetaNormal(cantArchivos, cantLetras, cantidadPalabras, sigma):
    fileName = "normal"
    #si no existe la carpeta la creamos
    if not os.path.exists("../data/normal/"):
        os.makedirs("../data/normal/")
        
    #borramos los archivos viejos para generar los nuevos
    for f in os.listdir("../data/normal/"):
        os.remove("../data/normal/"+ str(f) )

    for i in range(cantArchivos):
        nombre = fileName + str(i)
        generarArchivoNormal(nombre, cantLetras, cantidadPalabras, sigma)

In [7]:
def correr_experimento(funcion, cant_threads, archivos_entradas):
    # Crear proceso para ejecutar el codigo.
    taskAndParameters = [ "../build/archivos", cant_threads, str(len(archivos_entradas))]
    
    for f in archivos_entradas:
        taskAndParameters.append( os.path.abspath(f) )

    process = subprocess.Popen(taskAndParameters, stderr=subprocess.PIPE, stdout=subprocess.PIPE, universal_newlines = True)

    # Correr experimento.
    exit_code = process.wait()

    # Verificar que el proceso no fallo.
    if exit_code != 0:
        print(exit_code)
        print(process.stderr.read())
        process.stderr.close()
        raise(F"Hubo un error en la experimentacion: {funcion} con la instancia {archivos_entradas}.")
    # Leer salida de STDERR con los tiempos de ejecucion de cada metodo.
    # [cant threads , cantidad archivos, tiempo promedio, maximoasignado, minimo asignado, media restante]
    resultado = process.stdout.readline()
    process.stdout.close()
    process.stderr.close()
       
    return resultado.split(" ")

In [8]:
def suma(s,t):
    res = [0.0]*len(s)
    for i in range( len(s) ):
        res [i] = float(s[i]) + float(t[i])
    return res
def promedio(s, repe):
    res = [0.0]*len(s)
    for i in range(len(s)):
        res[i] = s[i] / int(repe)
    return res

In [9]:
dir_paths = "../data/uniforme/"
entries = []
carpeta = "uniforme"

generarCarpetaUniformes( cantidadArchivos, 6, cantidadPalabrasPorArchivo )

file_paths = getAllFilesAsListOfStrings(dir_paths)

for t in range( 1, THREADS+1 ):
    clear_output(wait=True)
    display(F"Ejecutando carpetas {carpeta} con cantidad de Threads {t} " )
    salida_temp = [0.0]*5
    for i in range (ITERACIONES):
        salida = correr_experimento("archivos", str(t), file_paths )
        salida_temp = suma( salida_temp, salida )
    salida_temp = promedio(salida_temp, ITERACIONES)
    entries.append(salida_temp)

df_resultado = pd.DataFrame(entries, columns=["cantThreads", "cantArchivos", "tiempoTotal", "tiempoMedio", "medLoad"])
os.makedirs(F"results/{result_dir}/", exist_ok=True)

display(F"Guardando los resultados en el archivo results/{result_dir}/uniforme.csv")
df_resultado.to_csv(F"results/{result_dir}/uniforme.csv", index=False, header=True)


'Ejecutando carpetas uniforme con cantidad de Threads 8 '

'Guardando los resultados en el archivo results/archivos/uniforme.csv'

In [10]:
dir_paths = "../data/normal/"
entries = []
carpeta = "normal"

generarCarpetaNormal(cantidadArchivos, 6, cantidadPalabrasPorArchivo, 3)

file_paths = getAllFilesAsListOfStrings(dir_paths)

for t in range( 1, THREADS+1 ):
    clear_output(wait=True)
    display(F"Ejecutando carpetas {carpeta} con cantidad de Threads {t} " )
    salida_temp = [0.0]*5
    for i in range(ITERACIONES):
        salida = correr_experimento("CargarArchivos", str(t), file_paths)
        salida_temp = suma(salida_temp, salida)
    salida_temp = promedio(salida_temp, ITERACIONES)
    entries.append(salida_temp)

df_resultado = pd.DataFrame(entries, columns=["cantThreads", "cantArchivos", "tiempoTotal", "tiempoMedio", "medLoad"])
os.makedirs(F"results/{result_dir}/", exist_ok=True)

clear_output(wait=True)
display(F"Guardando los resultados en el archivo results/{result_dir}/normalsigma3.csv")
df_resultado.to_csv(F"results/{result_dir}/normalsigma3.csv", index=False, header=True)

'Guardando los resultados en el archivo results/archivos/normalsigma3.csv'

In [24]:
sigmas = ["0.125", "0.25", "0.5", "1", "2", "3", "4", "8"]
entries = []

dir_normal = "../data/normal/"

for sigma in sigmas:
    entries = []
    for i in range(100, 1001, 100):
        generarCarpetaNormal( cantidadArchivos, 10, i, float(sigma) ) #i: cantidad de palabras por letras [ 100, 200 ... 1000 ]
        file_paths = getAllFilesAsListOfStrings(dir_normal)
        clear_output(wait=True)
        salida_temp = [0.0]*5
        for j in range(3):
            salida = correr_experimento("CargarArchivos", str(4), file_paths)
            salida_temp = suma(salida_temp, salida)
        salida_temp = promedio(salida_temp, ITERACIONES)
        salida_temp.append(i)
        entries.append(salida_temp)
        df_resultado = pd.DataFrame(entries, columns=["cantThreads", "cantArchivos", "tiempoTotal","tiempoMedio", "medLoad", "cantPalabras"])
        os.makedirs( F"results/{result_dir}/", exist_ok=True )
    clear_output(wait=True)
    display(F"Guardando los resultados en el archivo results/{result_dir}/sigma{sigma}variandocantPalabras.csv")
    df_resultado.to_csv(F"results/{result_dir}/sigma{sigma}variandocantPalabras.csv", index=False, header=True)



'Guardando los resultados en el archivo results/archivos/sigma8variandocantPalabras.csv'

In [23]:
#sigmas = ["0.125","0.25", "0.375", "0.5","0.75","0.875", "1", "1.25", "1.5", "1.75", "2","2.25", "2.5","2.75", "3","3.25", "3.5", "4", "4.5", "5", "6", "7", "8"]
sigmas = ["0.125", "0.25", "0.375", "0.5","0.75" ,"1", "2", "3", "4", "8"]
entries = []

dir_normal = "../data/normal/"

for i in sigmas:
    generarCarpetaNormal(cantidadArchivos, 10, cantidadPalabrasPorArchivo, float(i) )
    file_paths = getAllFilesAsListOfStrings(dir_normal)
    clear_output(wait=True)
    display( F"Sigma = {i}" )
    salida_temp = [0.0]*5
    for j in range(ITERACIONES):
        salida = correr_experimento("CargarArchivos", str(4), file_paths)
        salida_temp = suma(salida_temp, salida)
    salida_temp = promedio(salida_temp, ITERACIONES)
    salida_temp.append(i)
    entries.append(salida_temp)
    df_resultado = pd.DataFrame(entries, columns=["cantThreads", "cantArchivos", "tiempoTotal", "tiempoMedio", "medLoad", "sigma"])
    os.makedirs( F"results/{result_dir}/", exist_ok=True )

clear_output(wait=True)
display(F"Guardando los resultados en el archivo results/{result_dir}/sigma.csv")
df_resultado.to_csv(F"results/{result_dir}/sigma.csv", index=False, header=True)

'Guardando los resultados en el archivo results/archivos/sigma.csv'